In [1]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass(frozen=True)
class ANNTrainingConfig:
    root_dir: Path
    data_file: Path
    model_save_dir: Path
    project_name: str

    # Data parameters
    sample_size: int
    target_column: str

    # Training parameters
    epochs_tuning: int
    epochs_final: int
    batch_size: int
    learning_rate: float

    # Hyperparameter tuning ranges
    max_trials: int
    num_layers_range: List[int]
    units_range: List[int]
    dropout_range: List[float]


In [2]:
from ANNClassifier.constants import *
from ANNClassifier.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_ann_training_config(self) -> ANNTrainingConfig:
        config = self.config.ann_training  # section in config.yaml
        params = self.params

        create_directories([config.model_save_dir])

        ann_training_config = ANNTrainingConfig(
            root_dir=Path(config.root_dir),
            data_file=Path(config.data_file),
            model_save_dir=Path(config.model_save_dir),
            project_name=config.project_name,

            sample_size=params.SAMPLE_SIZE,
            target_column=params.TARGET_COLUMN,

            epochs_tuning=params.EPOCHS_TUNING,
            epochs_final=params.EPOCHS_FINAL,
            batch_size=params.BATCH_SIZE,
            learning_rate=params.LEARNING_RATE,

            max_trials=params.MAX_TRIALS,
            num_layers_range=params.NUM_LAYERS_RANGE,
            units_range=params.UNITS_RANGE,
            dropout_range=params.DROPOUT_RANGE
        )

        return ann_training_config


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_ann_training_config(self) -> ANNTrainingConfig:
        config = self.config.ann_training
        params = self.params

        create_directories([config.model_save_dir])

        ann_training_config = ANNTrainingConfig(
            root_dir=config.root_dir,
            data_file=config.data_file,
            model_save_dir=config.model_save_dir,
            project_name=config.project_name,

            sample_size=params.SAMPLE_SIZE,
            target_column=params.TARGET_COLUMN,

            epochs_tuning=params.EPOCHS_TUNING,
            epochs_final=params.EPOCHS_FINAL,
            batch_size=params.BATCH_SIZE,
            learning_rate=params.LEARNING_RATE,

            max_trials=params.MAX_TRIALS,
            num_layers_range=params.NUM_LAYERS_RANGE,
            units_range=params.UNITS_RANGE,
            dropout_range=params.DROPOUT_RANGE
        )

        return ann_training_config

In [5]:
import os
import urllib.request as request
import tensorflow as tf

In [10]:
import tensorflow as tf
from pathlib import Path
# from src.bitcoin_forecaster.entity.config_entity import ANNTrainingConfig

class PrepareBaseModel:
    def __init__(self, config: ANNTrainingConfig, input_dim: int):
        self.config = config
        self.input_dim = input_dim

        # Ensure dropout range is a flat list of floats
        self.config.dropout_range = self._flatten_and_cast(self.config.dropout_range)

    @staticmethod
    def _flatten_and_cast(dropout_range):
        # Handle nested or stringified lists safely
        if isinstance(dropout_range, list):
            flat = []
            for d in dropout_range:
                if isinstance(d, list):
                    flat.extend(d)
                else:
                    flat.append(d)
            return [float(val) for val in flat]
        elif isinstance(dropout_range, str):
            # Handles string like "[0.1, 0.2, 0.3]"
            import ast
            return [float(x) for x in ast.literal_eval(dropout_range)]
        else:
            raise ValueError("Invalid format for dropout_range")

    def get_base_model(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.Input(shape=(self.input_dim,)))

        num_layers = max(self.config.num_layers_range)
        units = max(self.config.units_range)
        dropout = float(min(self.config.dropout_range))  # Ensure float type

        for _ in range(num_layers):
            self.model.add(tf.keras.layers.Dense(units, activation='relu'))
            self.model.add(tf.keras.layers.Dropout(dropout))

        self.model.add(tf.keras.layers.Dense(1, activation='linear'))  # Regression output

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=float(self.config.learning_rate)),
            loss='mean_squared_error',
            metrics=['mean_absolute_error']
        )

        self.save_model(path=self.config.model_save_dir / "base_model.h5", model=self.model)

    def update_base_model(self):
        self.full_model = self.model
        self.save_model(path=self.config.model_save_dir / "updated_model.h5", model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [12]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'